In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Thank the smart people at the below link for this code:
https://medium.com/analytics-vidhya/the-shapley-value-approach-to-multi-touch-attribution-marketing-model-e345b35f3359



**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
from pandasql import sqldf
import itertools
from collections import defaultdict
from itertools import permutations,combinations


**Ingest Data**

In [ ]:
data = pd.read_csv('/kaggle/input/marketing-campaign/marketing.csv')
data.dropna(axis = 0, inplace = True)
data["conversion"]=data["converted"].astype('int')
data.to_csv('marketing-clean.csv',index=False)

**Reduce Features to Channels / Conversions and dates**

In [ ]:

data = data[['user_id','date_served','marketing_channel','conversion']]

data

In [ ]:
#set global funtions
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
#SQL code for coalitions

q = """SELECT channels_subset, sum(b.conversion) as conversion_sum
FROM(
SELECT user_id, GROUP_CONCAT(DISTINCT(marketing_channel)) as channels_subset, max(conversion) as conversion
FROM(
SELECT user_id,marketing_channel,conversion
FROM data
ORDER BY user_id,marketing_channel
) a
GROUP BY user_id)
b
GROUP BY channels_subset;"""

names = pysqldf(q)
names

Create a function that returns all possible combination of the channel


In [ ]:
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in itertools.combinations(List, i+1)]
    return PS

Return all the possible subsets of a set of channels


In [ ]:
def subsets(s):
    '''
    This function returns all the possible subsets of a set of channels.
    input :
            - s: a set of channels.
    '''
    if len(s)==1:
        return s
    else:
        sub_channels=[]
        for i in range(1,len(s)+1):
            sub_channels.extend(map(list,itertools.combinations(s, i)))
    return list(map(",".join,map(sorted,sub_channels)))

Computes the worth of each coalition.


In [ ]:
def v_function(A,C_values):
    '''
    This function computes the worth of each coalition.
    inputs:
            - A : a coalition of channels.
            - C_values : A dictionnary containing the number of conversions that each subset of channels has yielded.
    '''
    subsets_of_A = subsets(A)
    #print(subsets_of_A)
    #exit()
    worth_of_A=0
    for subset in subsets_of_A:
        #print("subset:", subset)
        if subset in C_values:
            #print("subset:", subset, "; Value:", C_values[subset])
            worth_of_A += C_values[subset]
    return worth_of_A

Calculate the factorial of a number (a non-negative integer)


In [ ]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

Final code here


In [ ]:
def calculate_shapley(df, col_name):
    '''
    This function returns the shapley values
            - df: A dataframe with the two columns: ['channels_subset', 'conversion_sum'].
            The channel_subset column is the channel(s) associated with the conversion and the count is the sum of the conversions. 
            - col_name: A string that is the name of the column with conversions
            **Make sure that that each value in channel_subset is in alphabetical order. Email,PPC and PPC,Email are the same 
            in regards to this analysis and should be combined under Email,PPC.
            ***Be careful with the distinct number of channels because this can signifcantly slow the perfomance of this function.
            Remember the total possible combinations of channels is 2^(n) where n is the number of channels. If you have 30 channels
            this is 1,073,741,824 combinations. 
            
    '''
    c_values = df.set_index("channels_subset").to_dict()[col_name]
    df['channels'] = df['channels_subset'].apply(lambda x: x if len(x.split(",")) == 1 else np.nan)
    channels = list(df['channels'].dropna().unique())
    
    v_values = {}
    for A in power_set(channels):
        v_values[','.join(sorted(A))] = v_function(A,c_values)
    #print(v_values)
    n=len(channels)
    shapley_values = defaultdict(int)

    for channel in channels:
        for A in v_values.keys():
            #print(A)
            if channel not in A.split(","):
                #print(channel)
                cardinal_A=len(A.split(","))
                A_with_channel = A.split(",")
                A_with_channel.append(channel)            
                A_with_channel=",".join(sorted(A_with_channel))
                # Weight = |S|!(n-|S|-1)!/n!
                weight = (factorial(cardinal_A)*factorial(n-cardinal_A-1)/factorial(n))
                # Marginal contribution = v(S U {i})-v(S)
                contrib = (v_values[A_with_channel]-v_values[A]) 
                shapley_values[channel] += weight * contrib
        # Add the term corresponding to the empty set
        shapley_values[channel]+= v_values[channel]/n 
        
    return shapley_values

# Shapley Value Attribution

In [ ]:
#visualizations of Shapley attribution
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns
result = pd.DataFrame(list(dict(calculate_shapley(names, "conversion_sum")).items()),columns = ['Channel','Contributions'])
# Visualization
plt.subplots(figsize=(18, 6))
sns.barplot(x='Channel', y='Contributions', data=result)
plt.show()

In [ ]:
calculate_shapley(names, "conversion_sum")

# SIMPLE Attribution

In [ ]:
simple_attr = pd.pivot_table(data, values='conversion', index=['marketing_channel'], aggfunc=np.sum)
simple_attr = simple_attr.reset_index()
simple_attr

In [ ]:
# Visualization of simple attributions
plt.subplots(figsize=(18, 6))
sns.barplot(x='marketing_channel', y='conversion', data=simple_attr)
plt.show()